<a href="https://colab.research.google.com/github/kovaciardey/ce888-repo/blob/main/Lab2/bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook you'll create your own bootstrap function following the bootstrap algorithm (check the lecture notes!)

In [ ]:
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np



In [ ]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab2%20-%20bootstrap/customers.csv')
data = df.values.T[1]

In [ ]:
# Checking the notes from the lecture, create here your own bootstrap function:
# 1. Sample from the input array x to create an array of samples of shape (n_bootstraps, sample_size)
# Hint: Check the function random.choice() on Numpy
# 2. Calculate and save the mean of the array (this is "data_mean" that is returned by the function)
# 3. Calculate the mean from each bootstrap (i.e., row) and store it.
# (This should be an array of n_bootstraps values)
# 4. Calculate the lower and upper bounds for a 95% CI (hint: check the percentile function on Numpy)
# 5. Return data_mean, and the lower and upper bounds of your interval
def bootstrap_mean(x, sample_size, n_bootstraps):
    # <---INSERT YOUR CODE HERE--->
    bootstrap_means = np.empty((n_bootstraps))
    data_array = np.empty((n_bootstraps, sample_size))

    for i in range(n_bootstraps):
        data_array[i] = np.random.choice(x, size=sample_size)
        bootstrap_means[i] = np.mean(data_array[i])

    data_mean = np.mean(data_array)
    lower, upper = np.percentile(bootstrap_means, 2.5), np.percentile(bootstrap_means, 97.5)

    return data_mean, lower, upper


# for i in range(100, 50000, 1000):
#     boot = bootstrap_mean(data, data.shape[0], i)

In [ ]:
# Call your bootstrap function and plot the results

iterations = 10000
# iterations = 50000

boots = []
for i in range(100, iterations, 1000):
    boot = bootstrap_mean(data, data.shape[0], i)

    print(boot)

    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Bootstrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, iterations)



Now, modify the bootstrap function you created above so that you can pass your desired confidence interval as a parameter.



In [ ]:
def bootstrap_mean_ci(sample, sample_size, n_bootstraps, ci):
    # <---INSERT YOUR CODE HERE--->
    bootstrap_means = np.empty((n_bootstraps))
    data_array = np.empty((n_bootstraps, sample_size))

    for i in range(n_bootstraps):
        data_array[i] = np.random.choice(sample, size=sample_size)
        bootstrap_means[i] = np.mean(data_array[i])

    lower_ci_percentile = (100 - ci) / 2.0
    upper_ci_percentile = 100 - lower_ci_percentile

    data_mean = np.mean(data_array)
    lower, upper = np.percentile(bootstrap_means, lower_ci_percentile), np.percentile(bootstrap_means, upper_ci_percentile)

    return data_mean, lower, upper

def bootstrap_standard_deviation_ci(sample, sample_size, n_bootstraps, ci):
    # <---INSERT YOUR CODE HERE--->
    bootstrap_means = np.empty((n_bootstraps))
    data_array = np.empty((n_bootstraps, sample_size))

    for i in range(n_bootstraps):
        data_array[i] = np.random.choice(sample, size=sample_size)
        bootstrap_means[i] = np.std(data_array[i])

    lower_ci_percentile = (100 - ci) / 2.0
    upper_ci_percentile = 100 - lower_ci_percentile

    data_mean = np.mean(data_array)
    lower, upper = np.percentile(bootstrap_means, lower_ci_percentile), np.percentile(bootstrap_means, upper_ci_percentile)

    return data_mean, lower, upper

In [ ]:
boots = []
for i in range(100, 10000, 1000):
    # boot = bootstrap_mean_ci(data, data.shape[0], i, 80)
    boot = bootstrap_standard_deviation_ci(data, data.shape[0], i, 85)

    print(boot)

    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Boostrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 10000)

#sns_plot.savefig("bootstrap_confidence_80.pdf", bbox_inches='tight')


# Vehicles dataset

Now let's work on a different dataset, which is stored in the vehicles.csv file.


In [ ]:
# Load and visualise the vehicles dataset
# To load the dataset: https://neptune.ai/blog/google-colab-dealing-with-files (check section "Load individual files directly from GitHub")


# Note that the current and new fleets are in different columns and have different lengths, so bear this in mind when you're plotting.
# You can create separate scatterplots for the two fleets, as you would with the histograms, 
# or plot them both in one plot (but not one against the other).
# <---INSERT YOUR CODE HERE--->
# Note: you can add more cells as needed to organise your code and your plots

vehicles_df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab2%20-%20bootstrap/vehicles.csv')
cf_data = vehicles_df["Current fleet"]
nf_data = vehicles_df["New Fleet"]

nf_data = nf_data[np.logical_not(np.isnan(nf_data))]

# print(cf_data)
# print(nf_data)

sns.displot(cf_data, bins=20, kde=False, rug=True)
sns.displot(nf_data, bins=20, kde=False, rug=True)


## Compare the two fleets

The business analysts come up a comparison algorithm that requires the upper and lower bounds for the mean in order to say which fleet is better.
1. Calculate the mean of both samples.
2. Using the bootstrap function that you created:
    - Construct the 95% CI of the mean of the current fleet.
    - Construct the 95% CI of the mean of the new fleet.
    - Are they comparable? (i.e., is one better than the other?) -- you can do this with a permutation test (check the lecture notes!)

In [ ]:
# <---INSERT YOUR CODE HERE--->
boots = []
for i in range(100, 10000, 1000):
    boot = bootstrap_mean_ci(cf_data, cf_data.shape[0], i, 95)

    print(boot)

    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Boostrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 10000)

In [ ]:
boots = []
for i in range(100, 10000, 1000):
    boot = bootstrap_mean_ci(nf_data, nf_data.shape[0], i, 95)

    print(boot)

    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Boostrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 10000)

In [ ]:
# Create your own function for a permutation test here (you will need it for the lab quiz!):

def permut_test(sample1, sample2, n_permutations):
    """
    sample1: 1D array
    sample2: 1D array (note that the size of the two arrays can be different)
    n_permutations: number of permutations to calculate the p-value
    """
    arr1_size = len(sample1)
    arr2_size = len(sample2)

    old = sample1
    new = sample2

    # calculate observed value of the test statistic
    old_mean = np.mean(old)
    new_mean = np.mean(new)

    tObs = new_mean - old_mean

    print("tobs = ", tObs)

    counter = 0
    for i in range(n_permutations):
        concat = np.concatenate((old, new))
        perm = np.random.permutation(concat)

        pold = perm[:arr1_size]
        pnew = perm[arr1_size:]

        # print(perm)
        # print(pold)
        # print(pnew)

        tPerm = np.mean(pnew) - np.mean(pold)
        print("tPerm = ", tPerm)

        if tPerm > tObs:
            counter += 1

    print(counter)
    pvalue = counter / n_permutations 

    return pvalue


print("p value = ", permut_test(cf_data, nf_data, 10))


tobs =  10.336434344974837
tPerm =  0.39855625031772846
tPerm =  0.46525341873824644
tPerm =  -0.3684611865182248
tPerm =  1.5657566976767825
tPerm =  0.4986020029485019
tPerm =  -1.4689644654567644
tPerm =  1.3823394845203616
tPerm =  1.9992882924101494
tPerm =  1.1488993950485487
tPerm =  -0.7186213207259442
0
p value =  0.0
